In [47]:
import numpy as np
import scipy.integrate as intgr
import pandas as pd  
from google.colab import files

In [48]:
def equations(X, t, r, l, omega, theta_1, theta_2):
  phi, eta = X
  d_phi = eta
  g = 9.8     # Constant

  # Calculate components separately
  # Coefficient (Constant)
  coeff = r * omega * (theta_1 - theta_2) / 2

  # Theta function
  theta_t = (theta_1 - theta_2) / 2 * np.cos(omega*t) + (theta_1 + theta_2) / 2
  d_theta_t = -omega * (theta_1 - theta_2) / 2 * np.sin(omega*t)

  # d_eta in two parts
  first = 1/l * (coeff * (np.sin(theta_t) * np.sin(omega*t) * d_theta_t - omega * np.cos(theta_t) * np.cos(omega*t) - g)) * np.sin(phi)
  second = 1/l * coeff * (np.cos(theta_t) * np.sin(omega*t) * d_theta_t + omega * np.sin(theta_t) * np.cos(omega*t)) * np.cos(phi)

  # Lagrange's Equation 
  d_eta = first - second
  
  return [d_phi, d_eta]


# r: oscillation radius, l: length of rod, phi: angle of mass movement
# theta_1 & theta_2: max/min angle for oscillation
def init(r=0.05, l=0.05, omega=188.5, theta_1=3*np.pi/4, theta_2=np.pi/4, phi0=0.1, eta0=0.0, T=10):
  t = np.arange(0, T, 0.01)    # time points to evaluate solution at
  sol = intgr.odeint(equations, [phi0, eta0], t, args=(r, l, omega, theta_1, theta_2))
  phi = sol[:,0]
  return r, l, omega, phi, theta_1, theta_2, t

In [49]:
base_theta = [1/4, 1/2, 3/4] # Initial position of base (from 1/4 pi to 3/4 pi)
theta_pp = [1/10, 1/8, 1/6, 1/4, 1/2] # Peak-to-peak angle oscillation of base (from 1/10 pi to 1/2 pi)

for b_t in base_theta:
  for t_pp in theta_pp:
    file_name = "Base theta %.2f pi, Theta_pp %.2f pi" %(b_t, t_pp)
    ang_1 = b_t*np.pi + t_pp*np.pi/2
    ang_2 = b_t*np.pi - t_pp*np.pi/2
    r, l, omega, phi, theta_1, theta_2, t = init(theta_1=ang_1, theta_2=ang_2)
      
    theta_t = (theta_1 - theta_2) / 2 * np.cos(omega*t) + (theta_1 + theta_2) / 2
    x = l * np.sin(phi) + r * np.cos(theta_t) 
    y = l * np.cos(phi) + r * np.sin(theta_t)   

    x = np.round(x, 4)
    y = np.round(y, 4)
    phi = np.round(phi, 4)

    values = {'x': x, 'y': y, '\u03C6': phi}
    df = pd.DataFrame(values) 
    df.to_csv(file_name + '.csv') 
    #files.download(file_name + '.csv')   # Download files